<img src="./images/DLI_Header.png" style="width: 400px;">

# 4. Llama2 deployment with NVIDIA TensorRT-LLM and Triton Inference server 

In the previous section, we have seen how to take an unoptimized model and significantly accelerate it using TensorRT-LLM. This is significant, as it allows us to dramatically reduce the size of the capital investment needed to deploy an application as well as the ongoing operational costs. TensorRT-LLM can frequently be the difference between a non-feasible idea and a product that generates revenue. At the end of the previous notebook, we only had a binary with no real means of serving it to large groups of users. We did not have a way to deploy this binary to production, not to mention, to manage the efficiency of its execution. We will address the above and have an application that is close to being ready for production. 

In this notebook we will: 
* Discuss key features of NVIDIA Triton inference server and how it can be integrated with the TensorRT-LLM Execution Backend. 
* Learn how to prepare the LLama2 for deployment using Triton. 
* Run Triton inference server with Llama2 using a two-way Tensor Parallel deployment. 
* We will write a client application using our model hosted on Triton using Python client API `tritonclient` 


**[4.1 NVIDIA Triton Inference Server](#4.1)<br>** 
**[4.2 Overview of the end-to-end inference pipeline](#4.2)<br>** 
**[4.3 Preparing the inference environment](#4.3)<br>** 
**[4.4 Triton Inference Server configuration](#4.4)<br>** 
**[4.5 Launching Triton Inference Server](#4.5)<br>** 
**[4.6 Writing client application consuming our LLama2 model](#4.6)<br>** 

# 4.1 NVIDIA Triton Inference Server

NVIDIA Triton™ Inference Server, part of the NVIDIA AI platform, is an open-source inference serving software that helps standardize model deployment and execution and delivers fast and scalable AI in production. Triton streamlines AI inference by enabling teams to deploy, run, and scale trained AI models from any framework on any GPU- or CPU-based infrastructure. It provides AI researchers and data scientists the freedom to choose the right framework for their projects without impacting production deployment. It also helps developers deliver high-performance inference across cloud, on-prem, edge, and embedded devices. 
<div style="text-align:center"> 
<img src="./images/TRTLLM_custom_backend.PNG" style="width: 60%;position:relative;" /> 
</div> 

Triton inference server is distributed with a number of execution backends including backends supporting TensorRT, PyTorch, custom Python/C++ backend, HugeCTR, OpenVion and many other including TensorRT-LLM. It can be extended by developers as new backends can also be integrated. [TensorRT-LLM execution backend](https://github.com/triton-inference-server/tensorrtllm_backend) written by NVIDIA allows to deploy models optimized using TensorRT-LLM library not only in a single GPU but also multi-GPU and multi node mode. 

# 4.2 Overview of the end-to-end inference pipeline

Before we dive into the Triton deployment process, let us take a bird's eye view of the end-to-end deployment process. We just need now to focus on configuring Triton Inference server and its execution. 

<div style="text-align:center">
<img src="./images/TRTLLM_Triton_pipeline_nospell.png" style="width: 70%;position:relative;" />
</div>

# 4.3 Preparing the inference environment

We already downloaded `tensorrtllm_backend` code from GitHub (https://github.com/triton-inference-server/tensorrtllm_backend), so we will be able to use Triton's configs for LLama2 prepared by NVIDIA. They are placed in <code>tensorrtllm_backend/all_models/inflight_batcher_llm</code>. 

This configuration file is a perfect example of model ensemble in Triton Inference Server. Triton is a general purpose inference server allowing not only to run individual neural networks, but also construct complex ensembles/pipelines. These pipelines may contain multiple neural network stages connected with each other into one inference service. Pipelines can be composed of different components. One can add Python/C++ scripts before and/or after any neural network to add any pre/post processing steps that could transform the data into its final format.  </br> 

In the `LLama2` inference pipeline we have three distinct execution steps: </br> `preprocessing` -> `model execution with tensorrtllm` -> `postprocessing` </br>  

* preprocessing: tokenize the input text (Python Backend)

* tensorrtllm: infer the TRTLLM engine (TensorRT-LLM backend)

* postprocessing: decode the text output (Python Backend)

* ensemble folder describing the Inputs/Outputs and the sequence of models to call during inference.

When querying the TensorRT-LLM model, we will query only the "ensemble" which is responsible for all the pipeline. Overall, the client-server inference scheme looks like this: 

<div style="text-align:center">
<img src="./images/Triton_execution.png" style="width: 30%;position:relative;" />
</div>

In [ ]:
# Let's copy all the template congifuration files in our own "Model_repository" folder
%cd tensorrtllm_backend/all_models/inflight_batcher_llm/
!cp -r ensemble preprocessing tensorrt_llm postprocessing /dli/task/model_repository   

In [ ]:
# Copy our 4 GPUs TRTLLM engines to the prepared Triton model_repository/tensorrt_llm/1/
%cd /dli/task
!cp trt-engines/llama_13b/fp16/4-gpus/* model_repository/tensorrt_llm/1

# 4.4 Triton Inference Server configuration

#### Updating model configurations

**TODO**: Let's edit the Triton configuration files responsible for the Llama2 model inference. <br/> 

Start with the config file in the TensorRT-LLM repository: <b>[model_repository/tensorrt_llm/config.pbtxt](model_repository/tensorrt_llm/config.pbtxt)</b> <br/>

1) Please update `gpt_model_path` value with the path to our TensorRT-LLM engines (4GPUs) in the model_repository.<br/> 
parameters: {<br/> 
  key: "gpt_model_path"<br/> 
  value: {<br/> 
    string_value: "<<< FIXME >>>"<br/> 
  }<br/> 
}<br/> 

2) Please update the `model_transaction_policy` decoupled to "False".<br/> 
model_transaction_policy {<br/> 
  decoupled: "<<< FIXME >>>"<br/> 
}<br/> 

We also need to define the Tokenizer repository in the preprocessing and post processing step of the ensemble.<br/>
Let's modify the config file for the preprocessing step <b>[model_repository/preprocessing/config.pbtxt](model_repository/preprocessing/config.pbtxt)</b> <br/>

1) Please update the `tokenizer_dir` to the Llama tokenizer <br/>
parameters {<br/> 
  key: "tokenizer_dir"<br/> 
  value: {<br/> 
    string_value: "<<< FIXME >>>" <br/> 
  }<br/> 
}<br/> 

2) Please update the `tokenizer_type` to "llama" <br/>
parameters {<br/> 
  key: "tokenizer_type"<br/> 
  value: {<br/> 
    string_value: "<<< FIXME >>>" <br/> 
  }<br/> 
}<br/> 

Finally let's modify the config file for the postprocessing step <b>[model_repository/postprocessing/config.pbtxt](model_repository/postprocessing/config.pbtxt)</b><br/>

1) Please update the `tokenizer_dir` to the Llama tokenizer <br/>
parameters {<br/> 
  key: "tokenizer_dir"<br/> 
  value: {<br/> 
    string_value: "<<< FIXME >>>" <br/> 
  }<br/> 
}<br/> 

2) Please update the `tokenizer_type` to "llama" <br/>
parameters {<br/> 
  key: "tokenizer_type"<br/> 
  value: {<br/> 
    string_value: "<<< FIXME >>>" <br/> 
  }<br/> 
}<br/> 


If you have a doubt, please check the solutions files :
- [tensorrt_llm](solutions/config_tensorrtllm.pbtxt)
- [preprocessing](solutions/config_preprocessing.pbtxt)
- [postprocessing](solutions/config_postprocessing.pbtxt)



# 4.5 Launching Triton Inference Server

Let's now bring all the parts together and start Triton hosting our LLama2 inference pipeline: </br> 
1. We have built and started Triton docker container with the TensorRT-LLM backend that supports distributed inference of Llama</br> 
2. We have downloaded model weights and converted them into TensorRT-LLM format. </br> 
3. We have prepared the Triton configuration file 'config.pbtxt'.</br> 

We are ready to start Triton Inference Server! 

The command that we will launch is described in the python script [launch_triton_server.py](https://github.com/triton-inference-server/tensorrtllm_backend/blob/main/scripts/launch_triton_server.py).</br>
It is this command line, adapted for multiple GPU serving. </br>
`mpirun --allow-run-as-root -n 1 tritonserver --grpc-port 8001 --metrics-port 8002 --model-repository=/dli/task/model_repository &`</br>
Let us look at the command segment one at a time:
-  <code>mpirun --allow-run-as-root -n 1 </code> We uses `mpirun``, a command that will allow us to execute Triton on multiple GPUs. In this case, the parameter `-n`` indicates that the number of gpus is set to 1, meaning only one gpu will be used. One could specify a higher number, deploying across multiple servers in our datacenter. 
- <code>--http-port 8000 --grpc-port 8001 --metrics-port=8002 </code>open the HTTP (8000) and GRPC (8001) ports to handle requests and port 8002 to log the metrics 
- <code>--model-repository=some_path</code>, which points to the directory where we have placed configuration files for all models that Triton will be serving (Triton can easily serve multiple models/pipelines). Triton will automatically scan through this folder, find model configurations, load them onto the GPUs

<b>We will start Triton in an external Terminal window. All output logs will be printed into the external command window output. In a Jupyter notebook, this would stop at this command until Triton is terminated.</b></br>
If Triton starts successfully, we will see these lines in the output. </br>
It will display information about the ensemble of Llama stages that were found by the Triton in our model directory: </br>
<code>
+-------------------+---------+--------+</br>
| Model             | Version | Status |</br>
+-------------------+---------+--------+</br>
| ensemble          | 1       | READY  |</br>
| tensorrtllm       | 1       | READY  |</br>
| postprocessing    | 1       | READY  |</br>
| preprocessing     | 1       | READY  |</br>
+-------------------+---------+--------+</br></code>
</br>
It will also display information about successful service initialization:</br>
<code>
I0503 17:26:25.226719 1668 grpc_server.cc:4421] Started GRPCInferenceService at 0.0.0.0:8001</br>
I0503 17:26:25.227017 1668 http_server.cc:3113] Started HTTPService at 0.0.0.0:8000</br>
I0503 17:26:25.283046 1668 http_server.cc:178] Started Metrics Service at 0.0.0.0:8002</code>


To launch the triton server, we will run this python script from a terminal. </br>
<b>python tensorrtllm_backend/scripts/launch_triton_server.py --world_size=4 --model_repo=./model_repository </b>

**TODO**
1) Please exectute the Cell below and click on the Terminal link.
2) Then Copy and Paste the above Python command line in the terminal 

In [ ]:
 %%html
<pre>
   Step 1: Open a terminal session by following the <a href="", data-commandlinker-command="terminal:create-new">Terminal link</a>
   Step 2: Launch Triton with this command: <font color="green">python tensorrtllm_backend/scripts/launch_triton_server.py --world_size=4 --model_repo=./model_repository </font>
</pre>

# 4.6 Writing client applications consuming our Llama model

In this section, we will go through several methods to send requests to our Triton Server. </br>
4.6.1 will use `Curl` and `Generate endpoint` to send prompts requests through HTTP </br>
4.6.2 will use an example of existing Python Client script </br>
4.6.3 will demonstrate how to create your own python Client script </br>

## 4.6.1 Query the server with the Triton generate endpoint
Starting with Triton 23.10 release, you can query the server using Triton's [generate endpoint](https://github.com/triton-inference-server/server/blob/main/docs/protocol/extension_generate.md) with a curl command based on the following general format within your client environment/container:</br>
<code>curl -X POST localhost:8000/v2/models/${MODEL_NAME}/generate -d '{"{PARAM1_KEY}": "{PARAM1_VALUE}", ... }'</code></br>

In the case of the models used in this example, you can replace MODEL_NAME with ensemble. Examining the ensemble model's config.pbtxt file, we can see that 4 parameters are required to generate a response for this model:
- "text_input": Input text to generate a response from</br>
- "max_tokens": The number of requested output tokens</br>
- "bad_words": A list of bad words (can be empty)</br>
- "stop_words": A list of stop words (can be empty)</br>

Therefore, we can query the server in the following way:</br>
<code>curl -X POST localhost:8000/v2/models/ensemble/generate -d '{"text_input": "Answer the question: What is machine learning?", "max_tokens": 32, "bad_words": "", "stop_words": ""}'</code></br>
if using the ensemble model. 

Which should return a result similar to (formatted for readability):</br>
{</br>
  "model_name": "ensemble",</br>
  "model_version": "1",</br>
  "sequence_end": false,</br>
  "sequence_id": 0,</br>
  "sequence_start": false,</br>
  "text_output": "Answer the question: What is machine learning?\nMachine learning is a branch of artificial intelligence that uses statistical techniques to give computer systems the ability to “learn” (i.e., progressively improve performance"\n\n"</br>
}

In [ ]:
!curl -X POST localhost:8000/v2/models/ensemble/generate -d '{"text_input": "Answer the question: What is machine learning?", "max_tokens": 32, "bad_words": "", "stop_words": ""}'

## 4.6.2 Utilize the provided client script to send a request
You can also send requests to the "ensemble" model with the provided [python client script](tensorrtllm_backend/tools/inflight_batcher_llm/end_to_end_streaming_client.py) as following, which is using the Tritonclient interface:

In [ ]:
!python tensorrtllm_backend/tools/inflight_batcher_llm/end_to_end_streaming_client.py --prompt 'Born in north-east France, Soyer trained as a' --output_len 128

## 4.6.3 Write your own client script
Let's see more in details how to send requests using tritonclient.</br>
To create an instance of the `client`: 

In [ ]:
import tritonclient.grpc as grpcclient
from tritonclient.utils import np_to_triton_dtype

import numpy as np
client = grpcclient.InferenceServerClient("localhost:8001",
                                           verbose=False)

We will then implement a function `prepare inputs` that prepares the necessary inputs to infer our Llama model, as expected and documented in the ensemble Config.pbtxt file. </br> 

In [ ]:
def prepare_tensor(name, input):
    t = grpcclient.InferInput(name, input.shape,
                              np_to_triton_dtype(input.dtype))
    t.set_data_from_numpy(input)
    return t

def prepare_inputs(prompt,desired_output_len):
    input0 = [[prompt]]
    input_data = np.array(input0).astype(object)
    output_len = np.ones_like(input0).astype(np.uint32) * desired_output_len
    bad_words_list = np.array([[""]], dtype=object)
    stop_words_list = np.array([[""]], dtype=object)
    streaming = [[False]]
    streaming_data = np.array(streaming, dtype=bool)

    inputs = [
        prepare_tensor("text_input", input_data),
        prepare_tensor("max_tokens", output_len),
        prepare_tensor("bad_words", bad_words_list),
        prepare_tensor("stop_words", stop_words_list)
    ]
    return inputs

Send the request to the server and get back the result: 

In [ ]:
prompt = "Once upon a time, "
desired_output_len = 128
inputs = prepare_inputs(prompt,desired_output_len)

In [ ]:
result = client.infer("ensemble", inputs)
output0 = result.as_numpy("text_output")
print(output0)

# 4.7 Measure Latency

In [ ]:
import time
execution_time = 0
num_iterations = 10
for _ in range(num_iterations):
    start = time.time()
    output = client.infer("ensemble", inputs)
    end = time.time()
    execution_time += end - start

In [ ]:
print("Average inference time of 128 tokens is:",
     1000 * (execution_time/float(num_iterations)), "ms")

On the 4xA100 setup this course was built with, the average inference time of 128 tokens is around 1.5s. 
So, we got an acceleration of around 2.6x, which is a starting point but far from being the best we can achieve with TRTLLM. 
With some advanced parameters like Inflight Batching, you could even reach 9x acceleration like in the plot below. 

Now you know how to run large models in production, and Triton Inference Server allows you to access your model from any device in the world! 

<div style="text-align:center">
<img src="./images/TRTLLM_performances.png" style="width: 60%;position:relative;"><br/>
</div>

<h2 style="color:green;">Congratulations!</h2>

Congratulations on finishing the course! If you're ready to put your skills to the test, please move on to the assessment.
